In [1]:
from ckks_package.ckks import *

In [2]:
N = 2**15  # Ring degree
n = 2**3  # Number of slots
L_boot = 19  # Maximal level during bootstrapping
q0 = 2**29  # Smallest modulus
p = 2**23  # Scaling factor outside of bootstrapping
delta = q0  # Scaling factor during bootstrapping

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L_boot, q0, p, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(91.9) operations.

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 3.+1.j, -1.-1.j,  1.+2.j,  2.-1.j,  2.+3.j,  2.+0.j, -2.-2.j,
        -0.-2.j]),
 array([ 1.-2.j, -1.+2.j,  3.+1.j,  3.-1.j, -1.-3.j, -1.+0.j, -1.+2.j,
         3.-2.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[7340032 - 4339088X^2048 + 9756427X^4096 + 2889662X^6144 + 6673097X^8192 - 2722510X^10240 + 4511364X^12288 - 7610695X^14336 + 5712123X^18432 + 1633608X^20480 - 3777666X^22528 + 5190186X^24576 - 711010X^26624 - 2671214X^28672 - 7615437X^30720 mod(2^581),
 6291456 + 5269279X^2048 + 2906274X^4096 - 1495108X^6144 - 2224366X^8192 + 3845545X^10240 - 18309035X^12288 - 843742X^14336 - 3145728X^16384 + 6565413X^18432 - 1203818X^20480 - 5459558X^22528 + 741455X^24576 - 322083X^26624 + 360943X^28672 - 480968X^30720 mod(2^581)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).,
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 2.99999923e+00+9.99999144e-01j -1.00000061e+00-9.99999366e-01j
  9.99999631e-01+2.00000014e+00j  1.99999929e+00-1.00000068e+00j
  2.00000052e+00+3.00000028e+00j  1.99999850e+00-1.72470709e-06j
 -1.99999798e+00-2.00000072e+00j  4.60932071e-07-1.99999993e+00j]
[ 0.9999989 -1.99999996e+00j -0.99999863+1.99999975e+00j
  3.00000179+9.99998404e-01j  2.99999884-9.99998074e-01j
 -1.0000006 -3.00000063e+00j -0.99999978-1.13386055e-06j
 -0.99999886+2.00000030e+00j  3.00000025-1.99999866e+00j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^23 (level 23 out of 24).


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.99999813-1.00000082e+00j -1.99999924+1.00000039e+00j
  4.00000142+2.99999854e+00j  4.99999813-1.99999875e+00j
  0.99999992-3.43892337e-07j  0.99999872-2.85856765e-06j
 -2.99999683-4.25496074e-07j  3.00000071-3.99999859e+00j]
[ 4.99999411-4.99999973e+00j  2.99999809-1.00000187e+00j
  1.00000275+7.00000289e+00j  4.99999464-4.99999589e+00j
  7.00000066-9.00000506e+00j -1.99999795-5.88912642e-07j
  5.99999983-1.99999782e+00j -3.99999689-6.00000192e+00j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt_boot = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt_boot))

print(ct.get_poly_precision(ct_boot, CKKS.sk))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^0 (level 0 out of 24).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^1 (level 1 out of 24).
[ 2.99565384e+00+9.99227617e-01j -9.98994742e-01-9.99951545e-01j
  9.98968631e-01+1.99896575e+00j  1.99718024e+00-9.97146092e-01j
  1.99691394e+00+2.99636961e+00j  1.99924704e+00-2.76521241e-03j
 -1.99721728e+00-1.99840478e+00j  5.72026251e-06-1.99698581e+00j]
10.198002165721377
